# Lesson 3: Practical Lab Session

**Estimator Properties - Hands-On Exercises**

This notebook contains practical exercises for exploring:
- Bias-variance tradeoff through simulation
- Confidence interval coverage properties
- Bootstrap methods and diagnostics
- Real-world A/B testing analysis

**Prerequisites**: Complete notebooks 01-05 for theoretical background

**Estimated time**: 2-3 hours

## Setup

Import required libraries and configure plotting style

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set random seed for reproducibility
rng = np.random.default_rng(2025)

# Configure plotting style
sns.set_style("whitegrid")
sns.set_palette("colorblind")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("✓ Setup complete")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
# print(f"SciPy version: {stats.__version__}")

---

## Lab Task 1: Bias-Variance Tradeoff Investigation

**Goal**: Compare MLE vs. unbiased estimator for variance

**Theory**: 
- MLE: $\hat{\sigma}^2_{\text{MLE}} = \frac{1}{n}\sum(X_i - \bar{X})^2$ (biased)
- Unbiased: $s^2 = \frac{1}{n-1}\sum(X_i - \bar{X})^2$ (unbiased)
- MSE = Bias² + Variance

**Question**: Which estimator has lower MSE for small samples?

In [ ]:
def simulate_variance_estimators(true_mu=0, true_sigma=2, n=20, n_sim=10000):
    """
    Simulate and compare variance estimators

    Parameters:
    -----------
    true_mu : float
        True population mean
    true_sigma : float
        True population standard deviation
    n : int
        Sample size
    n_sim : int
        Number of simulations

    Returns:
    --------
    dict : Results containing bias, variance, and MSE for each estimator
    """
    mle_estimates = []
    unbiased_estimates = []

    for _ in range(n_sim):
        # Generate sample
        X = rng.normal(true_mu, true_sigma, n)
        xbar = X.mean()

        # MLE estimator
        mle_var = np.sum((X - xbar)**2) / n
        mle_estimates.append(mle_var)

        # Unbiased estimator
        unbiased_var = np.sum((X - xbar)**2) / (n - 1)
        unbiased_estimates.append(unbiased_var)

    # Convert to arrays
    mle_arr = np.array(mle_estimates)
    unb_arr = np.array(unbiased_estimates)

    # Compute metrics
    true_var = true_sigma**2

    results = {
        'MLE': {
            'bias': mle_arr.mean() - true_var,
            'variance': mle_arr.var(),
            'mse': ((mle_arr - true_var)**2).mean()
        },
        'Unbiased': {
            'bias': unb_arr.mean() - true_var,
            'variance': unb_arr.var(),
            'mse': ((unb_arr - true_var)**2).mean()
        }
    }

    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Histogram
    axes[0].hist(mle_arr, bins=50, alpha=0.6, label='MLE', density=True)
    axes[0].hist(unb_arr, bins=50, alpha=0.6, label='Unbiased', density=True)
    axes[0].axvline(true_var, color='red', linestyle='--', linewidth=2, label='True σ²')
    axes[0].set_xlabel('Estimated Variance')
    axes[0].set_ylabel('Density')
    axes[0].set_title(f'Distribution of Variance Estimators (n={n})')
    axes[0].legend()

    # MSE comparison
    metrics = ['Bias²', 'Variance', 'MSE']
    mle_vals = [results['MLE']['bias']**2, results['MLE']['variance'], results['MLE']['mse']]
    unb_vals = [results['Unbiased']['bias']**2, results['Unbiased']['variance'], results['Unbiased']['mse']]

    x_pos = np.arange(len(metrics))
    width = 0.35

    axes[1].bar(x_pos - width/2, mle_vals, width, label='MLE', alpha=0.8)
    axes[1].bar(x_pos + width/2, unb_vals, width, label='Unbiased', alpha=0.8)
    axes[1].set_xticks(x_pos)
    axes[1].set_xticklabels(metrics)
    axes[1].set_ylabel('Value')
    axes[1].set_title('Estimator Comparison')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

    return results

In [ ]:
# Run simulation
print("Running variance estimator simulation (n=20, 10,000 iterations)...\n")
results = simulate_variance_estimators(true_mu=0, true_sigma=2, n=20, n_sim=10000)

print("\nSimulation Results:")
print("="*50)
for name, metrics in results.items():
    print(f"\n{name}:")
    for metric, value in metrics.items():
        print(f"  {metric:10s}: {value:.6f}")

print("\n" + "="*50)
print("Key Observation:")
if results['MLE']['mse'] < results['Unbiased']['mse']:
    print("MLE has lower MSE despite being biased!")
    print("This demonstrates the bias-variance tradeoff.")
else:
    print("Unbiased estimator has lower MSE.")

**Exercise**: Try different sample sizes (n=10, 50, 100) and observe how the tradeoff changes.

In [ ]:
# Your code here: Test with different sample sizes

---

## Lab Task 2: Confidence Interval Coverage Study

**Goal**: Compare Wald, Wilson, and Agresti-Coull intervals for proportions

**Theory**:
- **Nominal coverage**: The claimed confidence level (e.g., 95%)
- **Actual coverage**: Proportion of CIs that contain true parameter
- Good methods have actual ≈ nominal coverage

**Question**: Which method maintains proper coverage across different p and n?

In [ ]:
def coverage_study(n_sim=5000, alpha=0.05):
    """
    Compare coverage of different CI methods for proportions

    Parameters:
    -----------
    n_sim : int
        Number of simulations per (p, n) combination
    alpha : float
        Significance level

    Returns:
    --------
    dict : Coverage results for each method
    """
    p_values = [0.05, 0.1, 0.3, 0.5]
    n_values = [20, 50, 100, 200]
    methods = ['Wald', 'Wilson', 'Agresti-Coull']

    results = {method: np.zeros((len(p_values), len(n_values)))
               for method in methods}

    z = stats.norm.ppf(1 - alpha/2)  # ~1.96 for 95% CI

    for i, p in enumerate(p_values):
        for j, n in enumerate(n_values):
            coverage = {method: 0 for method in methods}

            for _ in range(n_sim):
                # Generate data
                X = rng.binomial(1, p, size=n)
                phat = X.mean()

                # Wald interval
                se_wald = np.sqrt(phat * (1 - phat) / n)
                wald_lo = phat - z * se_wald
                wald_hi = phat + z * se_wald
                coverage['Wald'] += (wald_lo <= p <= wald_hi)

                # Wilson interval
                denom = 1 + z**2 / n
                center = (phat + z**2 / (2*n)) / denom
                radius = z * np.sqrt(phat*(1-phat)/n + z**2/(4*n**2)) / denom
                wilson_lo = center - radius
                wilson_hi = center + radius
                coverage['Wilson'] += (wilson_lo <= p <= wilson_hi)

                # Agresti-Coull interval
                n_tilde = n + 4
                p_tilde = (X.sum() + 2) / n_tilde
                se_ac = np.sqrt(p_tilde * (1 - p_tilde) / n_tilde)
                ac_lo = p_tilde - z * se_ac
                ac_hi = p_tilde + z * se_ac
                coverage['Agresti-Coull'] += (ac_lo <= p <= ac_hi)

            # Store coverage proportions
            for method in methods:
                results[method][i, j] = coverage[method] / n_sim

    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for ax, method in zip(axes, methods):
        im = ax.imshow(results[method], cmap='RdYlGn', vmin=0.90, vmax=0.99, aspect='auto')
        ax.set_xticks(range(len(n_values)))
        ax.set_xticklabels(n_values)
        ax.set_yticks(range(len(p_values)))
        ax.set_yticklabels(p_values)
        ax.set_xlabel('Sample Size (n)')
        ax.set_ylabel('True Proportion (p)')
        ax.set_title(f'{method} Method\n({int((1-alpha)*100)}% nominal coverage)')

        # Add text annotations
        for i in range(len(p_values)):
            for j in range(len(n_values)):
                text = ax.text(j, i, f'{results[method][i, j]:.3f}',
                             ha="center", va="center", color="black", fontsize=9)

        plt.colorbar(im, ax=ax, label='Coverage Probability')

    plt.tight_layout()
    plt.show()

    return results

In [ ]:
# Run coverage study (this may take 1-2 minutes)
print("Running CI coverage study...")
print("Testing 4 proportions × 4 sample sizes × 5000 simulations = 80,000 datasets")
print("This will take approximately 1-2 minutes...\n")

coverage_results = coverage_study(n_sim=5000)

print("\n" + "="*70)
print("Key Findings:")
print("="*70)
print("1. Wald interval: Poor coverage, especially for extreme p and small n")
print("2. Wilson interval: Maintains near-nominal coverage across all scenarios")
print("3. Agresti-Coull: Good approximation to Wilson, slightly conservative")
print("\nRecommendation: Use Wilson or Agresti-Coull, NOT Wald!")

**Exercise**: What happens with p=0.01 and n=20? Try it!

In [ ]:
# Run coverage study (this may take 1-2 minutes)

---

## Lab Task 3: Bootstrap Confidence Intervals with Diagnostics

**Goal**: Apply bootstrap to estimate median and compare with parametric methods

**Theory**:
- Bootstrap resamples data with replacement
- Approximates sampling distribution of any statistic
- Useful when theoretical distribution is unknown or complex

**Question**: How well does bootstrap work for the median?

In [ ]:
def bootstrap_analysis(data, statistic=np.median, n_bootstrap=10000, alpha=0.05):
    """
    Comprehensive bootstrap analysis with diagnostics

    Parameters:
    -----------
    data : array-like
        Original sample data
    statistic : callable
        Function to compute on each bootstrap sample
    n_bootstrap : int
        Number of bootstrap resamples
    alpha : float
        Significance level

    Returns:
    --------
    dict : Bootstrap results including CIs and diagnostics
    """
    n = len(data)

    # Original statistic
    theta_hat = statistic(data)

    # Generate bootstrap samples
    bootstrap_stats = np.array([
        statistic(rng.choice(data, size=n, replace=True))
        for _ in range(n_bootstrap)
    ])

    # Bootstrap SE
    bootstrap_se = bootstrap_stats.std()

    # Different CI methods
    # 1. Percentile
    ci_percentile = np.percentile(bootstrap_stats, [100*alpha/2, 100*(1-alpha/2)])

    # 2. Normal approximation
    ci_normal = (theta_hat - stats.norm.ppf(1-alpha/2) * bootstrap_se,
                 theta_hat + stats.norm.ppf(1-alpha/2) * bootstrap_se)

    # 3. Basic bootstrap
    ci_basic = (2*theta_hat - np.percentile(bootstrap_stats, 100*(1-alpha/2)),
                2*theta_hat - np.percentile(bootstrap_stats, 100*alpha/2))

    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1. Bootstrap distribution
    axes[0, 0].hist(bootstrap_stats, bins=50, density=True, alpha=0.7, edgecolor='black')
    axes[0, 0].axvline(theta_hat, color='red', linestyle='--', linewidth=2, label='Original')
    axes[0, 0].axvline(ci_percentile[0], color='green', linestyle='--', label='95% CI')
    axes[0, 0].axvline(ci_percentile[1], color='green', linestyle='--')
    axes[0, 0].set_xlabel('Bootstrap Statistic')
    axes[0, 0].set_ylabel('Density')
    axes[0, 0].set_title('Bootstrap Distribution')
    axes[0, 0].legend()

    # 2. Q-Q plot
    stats.probplot(bootstrap_stats, dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title('Q-Q Plot vs. Normal')

    # 3. Convergence plot
    cumulative_means = np.cumsum(bootstrap_stats) / np.arange(1, n_bootstrap + 1)
    axes[1, 0].plot(cumulative_means, linewidth=0.8)
    axes[1, 0].axhline(theta_hat, color='red', linestyle='--', label='Original')
    axes[1, 0].set_xlabel('Number of Bootstrap Samples')
    axes[1, 0].set_ylabel('Cumulative Mean')
    axes[1, 0].set_title('Bootstrap Convergence')
    axes[1, 0].legend()
    axes[1, 0].set_xlim([0, n_bootstrap])

    # 4. CI comparison
    methods = ['Percentile', 'Normal', 'Basic']
    cis = [ci_percentile, ci_normal, ci_basic]

    for i, (method, ci) in enumerate(zip(methods, cis)):
        axes[1, 1].plot([ci[0], ci[1]], [i, i], 'o-', linewidth=2, markersize=8, label=method)

    axes[1, 1].axvline(theta_hat, color='red', linestyle='--', alpha=0.5, label='Point Estimate')
    axes[1, 1].set_yticks(range(len(methods)))
    axes[1, 1].set_yticklabels(methods)
    axes[1, 1].set_xlabel('Value')
    axes[1, 1].set_title('CI Methods Comparison')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print results
    print(f"Original statistic: {theta_hat:.4f}")
    print(f"Bootstrap SE: {bootstrap_se:.4f}")
    print(f"\n95% Confidence Intervals:")
    print(f"  Percentile:    [{ci_percentile[0]:.4f}, {ci_percentile[1]:.4f}]")
    print(f"  Normal approx: [{ci_normal[0]:.4f}, {ci_normal[1]:.4f}]")
    print(f"  Basic:         [{ci_basic[0]:.4f}, {ci_basic[1]:.4f}]")

    return {
        'statistic': theta_hat,
        'bootstrap_se': bootstrap_se,
        'ci_percentile': ci_percentile,
        'ci_normal': ci_normal,
        'ci_basic': ci_basic,
        'bootstrap_distribution': bootstrap_stats
    }

In [ ]:
# Load height data
print("Loading height/weight data...")
heights_df = pd.read_csv("../../../shared/data/heights_weights_sample.csv")
heights = heights_df['height_cm'].values

print(f"Dataset: {len(heights)} observations")
print(f"Mean height: {heights.mean():.2f}")
print(f"Median height: {np.median(heights):.2f}")
print(f"Std dev: {heights.std():.2f}")
print("\n" + "="*70)

In [ ]:
# Bootstrap analysis for median height
print("\n=== Bootstrap Analysis for Median Height ===")
print("Running 10,000 bootstrap resamples...\n")

results_median = bootstrap_analysis(heights, statistic=np.median, n_bootstrap=10000)

In [ ]:
# Bootstrap analysis for IQR
def iqr(x):
    """Interquartile range"""
    return np.percentile(x, 75) - np.percentile(x, 25)

print("\n\n=== Bootstrap Analysis for IQR ===")
print("Running 10,000 bootstrap resamples...\n")

results_iqr = bootstrap_analysis(heights, statistic=iqr, n_bootstrap=10000)

**Exercise**: Try bootstrapping other statistics like:
- 90th percentile
- Coefficient of variation (std/mean)
- Skewness

In [ ]:
# Your code here: Try other statistics

---

## Lab Task 4: Real Data Application - A/B Testing

**Goal**: Analyze A/B test data using multiple inference methods

**Scenario**: Two website designs (variants A and B) tested for click-through rates

**Data format**: The CSV contains aggregated data per user:
- `user_id`: Unique user identifier
- `variant`: Which design the user saw ('A' or 'B')
- `impressions`: Number of times the user saw the design
- `clicks`: Number of times the user clicked

**Question**: Is Design B significantly better than Design A?

In [ ]:
# Load A/B test data
print("Loading A/B test data...\n")
ab_data = pd.read_csv("../../../shared/data/ab_test_clicks.csv")

print("Data preview:")
print(ab_data.head(10))
print(f"\nData shape: {ab_data.shape}")
print(f"Columns: {list(ab_data.columns)}")
print(f"\nVariants: {ab_data['variant'].unique()}")

In [ ]:
# Calculate summary statistics from aggregated data
# Group A
group_A = ab_data[ab_data['variant'] == 'A']
n_A = group_A['impressions'].sum()  # Total impressions for A
clicks_A = group_A['clicks'].sum()   # Total clicks for A

# Group B
group_B = ab_data[ab_data['variant'] == 'B']
n_B = group_B['impressions'].sum()  # Total impressions for B
clicks_B = group_B['clicks'].sum()   # Total clicks for B

# Click-through rates
p_A = clicks_A / n_A
p_B = clicks_B / n_B

print("\n" + "="*70)
print("Summary Statistics:")
print("="*70)
print(f"Group A: {clicks_A}/{n_A} impressions clicked (p̂ = {p_A:.4f})")
print(f"Group B: {clicks_B}/{n_B} impressions clicked (p̂ = {p_B:.4f})")
print(f"Difference (B - A): {p_B - p_A:.4f}")
if p_A > 0:
    print(f"Relative lift: {((p_B - p_A) / p_A * 100):.1f}%")

In [ ]:
# Method 1: Wald CI for difference
def proportion_diff_ci_wald(n1, p1, n2, p2, alpha=0.05):
    """Wald CI for difference in proportions"""
    diff = p2 - p1
    se = np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
    z = stats.norm.ppf(1 - alpha/2)
    return (diff - z*se, diff + z*se)

ci_wald = proportion_diff_ci_wald(n_A, p_A, n_B, p_B)

print("\n" + "="*70)
print("Method 1: Wald Confidence Interval")
print("="*70)
print(f"95% CI for difference (B - A): [{ci_wald[0]:.4f}, {ci_wald[1]:.4f}]")
if ci_wald[0] > 0:
    print("✓ Difference is statistically significant (95% confidence)")
    print("  → Group B has higher click-through rate")
elif ci_wald[1] < 0:
    print("✓ Difference is statistically significant (95% confidence)")
    print("  → Group A has higher click-through rate")
else:
    print("✗ No significant difference detected at 95% confidence level")

In [ ]:
# Method 2: Bootstrap CI for difference
def bootstrap_diff_proportions_aggregated(group_A_df, group_B_df, n_bootstrap=10000, alpha=0.05):
    """
    Bootstrap CI for difference in proportions from aggregated data

    Parameters:
    -----------
    group_A_df, group_B_df : DataFrames with 'impressions' and 'clicks' columns
    n_bootstrap : int
        Number of bootstrap resamples
    alpha : float
        Significance level

    Returns:
    --------
    tuple : (CI, bootstrap differences)
    """
    # Expand aggregated data to individual binary outcomes
    def expand_group(df):
        outcomes = []
        for _, row in df.iterrows():
            # Add clicks (1s)
            outcomes.extend([1] * row['clicks'])
            # Add non-clicks (0s)
            outcomes.extend([0] * (row['impressions'] - row['clicks']))
        return np.array(outcomes)

    data_A = expand_group(group_A_df)
    data_B = expand_group(group_B_df)

    # Bootstrap resampling
    diffs = []
    for _ in range(n_bootstrap):
        boot_A = rng.choice(data_A, size=len(data_A), replace=True)
        boot_B = rng.choice(data_B, size=len(data_B), replace=True)
        diffs.append(boot_B.mean() - boot_A.mean())

    diffs = np.array(diffs)
    ci = np.percentile(diffs, [100*alpha/2, 100*(1-alpha/2)])
    return ci, diffs

print("\n" + "="*70)
print("Method 2: Bootstrap Confidence Interval")
print("="*70)
print("Running 10,000 bootstrap resamples...")

ci_bootstrap, boot_diffs = bootstrap_diff_proportions_aggregated(group_A, group_B)

print(f"95% CI for difference (B - A): [{ci_bootstrap[0]:.4f}, {ci_bootstrap[1]:.4f}]")
if ci_bootstrap[0] > 0:
    print("✓ Difference is statistically significant (95% confidence)")
    print("  → Group B has higher click-through rate")
elif ci_bootstrap[1] < 0:
    print("✓ Difference is statistically significant (95% confidence)")
    print("  → Group A has higher click-through rate")
else:
    print("✗ No significant difference detected at 95% confidence level")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Bootstrap distribution
axes[0].hist(boot_diffs, bins=50, density=True, alpha=0.7, edgecolor='black')
axes[0].axvline(p_B - p_A, color='red', linestyle='--', linewidth=2, label='Observed Difference')
axes[0].axvline(0, color='gray', linestyle=':', linewidth=1, label='No Difference')
axes[0].axvline(ci_bootstrap[0], color='green', linestyle='--', label='95% Bootstrap CI')
axes[0].axvline(ci_bootstrap[1], color='green', linestyle='--')
axes[0].set_xlabel('Difference in Proportions (B - A)')
axes[0].set_ylabel('Density')
axes[0].set_title('Bootstrap Distribution of Difference')
axes[0].legend()

# Right: CI comparison
methods = ['Wald', 'Bootstrap']
cis = [ci_wald, ci_bootstrap]

for i, (method, ci) in enumerate(zip(methods, cis)):
    axes[1].plot([ci[0], ci[1]], [i, i], 'o-', linewidth=2, markersize=8, label=method)

axes[1].axvline(p_B - p_A, color='red', linestyle='--', alpha=0.5, label='Observed')
axes[1].axvline(0, color='gray', linestyle=':', linewidth=1, label='No Difference')
axes[1].set_yticks(range(len(methods)))
axes[1].set_yticklabels(methods)
axes[1].set_xlabel('Difference in Proportions (B - A)')
axes[1].set_title('Confidence Intervals Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Statistical test: two-proportion z-test
from statsmodels.stats.proportion import proportions_ztest

counts = np.array([clicks_B, clicks_A])
nobs = np.array([n_B, n_A])

z_stat, p_value = proportions_ztest(counts, nobs)

print("\n" + "="*70)
print("Method 3: Two-Proportion Z-Test")
print("="*70)
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print(f"✓ Reject null hypothesis at α=0.05 (p={p_value:.4f} < 0.05)")
    print("  → Significant difference between groups")
else:
    print(f"✗ Fail to reject null hypothesis at α=0.05 (p={p_value:.4f} ≥ 0.05)")
    print("  → No significant difference detected")

### Final Recommendation

Based on the analysis using three different methods:
1. **Wald CI**: Traditional approach
2. **Bootstrap CI**: Distribution-free approach
3. **Z-test**: Hypothesis test

All methods should give consistent conclusions about whether the difference is statistically significant.

---

## Summary and Conclusions

In this lab, you have:

1. ✅ **Explored the bias-variance tradeoff** through simulation
   - Observed that MLE can have lower MSE despite being biased
   - Learned that the "best" estimator depends on the metric (bias vs. MSE)

2. ✅ **Compared CI coverage properties** for proportions
   - Discovered that Wald intervals have poor coverage
   - Learned that Wilson and Agresti-Coull methods are superior

3. ✅ **Applied bootstrap methods** with comprehensive diagnostics
   - Constructed CIs for median and IQR using bootstrap
   - Assessed bootstrap distribution quality with Q-Q plots
   - Verified bootstrap convergence

4. ✅ **Analyzed real A/B test data** using multiple methods
   - Compared Wald, bootstrap, and hypothesis testing approaches
   - Drew conclusions about statistical significance

### Key Takeaways

- **MSE is often more important than bias alone** when evaluating estimators
- **Not all CI methods are created equal** - coverage matters more than simplicity
- **Bootstrap is powerful but requires sufficient sample size** (n ≥ 30 typically)
- **Multiple inference methods should give consistent results** when assumptions are met

### Next Steps

- Review theoretical notebooks (01-05) for deeper understanding
- Complete exercises in material.md
- Apply these methods to your own datasets
- Proceed to Lesson 4: Hypothesis Testing